In [1]:
import pandas as pd
import json
import re
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
import pickle
import ast

from flask import jsonify,make_response
from flask import jsonify
import json 

from flask import Flask

# Disable warnings, set Matplotlib inline plotting and load Pandas package
import warnings
warnings.filterwarnings('ignore')



In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession


spark = SparkSession\
        .builder\
        .appName("Category_Classification_PySpark")\
        .config('spark.sql.warehouse.dir','file:///home/sudipto21048867/Pyspark/spark-warehouse')\
        .getOrCreate()

In [3]:
jsonDF = spark.read.json("/user/sudipto21048867/Llyods_data/BASE_Data/TransformedSummary.json" )

In [4]:
from pyspark.sql.types import FloatType
from pyspark.sql.types import StringType
from pyspark.sql.functions import UserDefinedFunction as udf

In [5]:
jsonDF.printSchema()

root
 |-- Apr-17: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- AccountId: long (nullable = true)
 |    |    |-- Amount: string (nullable = true)
 |    |    |-- BankTransactionCode: string (nullable = true)
 |    |    |-- BookingDateTime: string (nullable = true)
 |    |    |-- CreditDebitIndicator: string (nullable = true)
 |    |    |-- Status: string (nullable = true)
 |    |    |-- TransactionId: long (nullable = true)
 |    |    |-- TransactionInformation: string (nullable = true)
 |    |    |-- ValueDateTime: string (nullable = true)
 |-- Aug-17: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- AccountId: long (nullable = true)
 |    |    |-- Amount: string (nullable = true)
 |    |    |-- BankTransactionCode: string (nullable = true)
 |    |    |-- BookingDateTime: string (nullable = true)
 |    |    |-- CreditDebitIndicator: string (nullable = true)
 |    |    |-- Status: string (nullable = true)
 | 

In [6]:
jsonObject = jsonDF.rdd.flatMap(lambda row: row[0])

In [7]:
jsonObject.first()

Row(AccountId=32353522, Amount=u'{"Amount": "6.88", "Currency": "GBP"}', BankTransactionCode=u'{"Code":  "POS"," SubCode": }', BookingDateTime=u'2017-04-03T20:21:50-08:00', CreditDebitIndicator=u'Debit', Status=u'Booked', TransactionId=100801, TransactionInformation=u'1177 31MAR17 D , UBER BV , HELP.UBER.COM NL', ValueDateTime=u'2017-04-03T20:21:50-08:00')

In [8]:
TransactionDf = jsonObject.toDF()

In [9]:
TransactionDf.show(5)

+---------+--------------------+--------------------+--------------------+--------------------+------+-------------+----------------------+--------------------+
|AccountId|              Amount| BankTransactionCode|     BookingDateTime|CreditDebitIndicator|Status|TransactionId|TransactionInformation|       ValueDateTime|
+---------+--------------------+--------------------+--------------------+--------------------+------+-------------+----------------------+--------------------+
| 32353522|{"Amount": "6.88"...|{"Code":  "POS","...|2017-04-03T20:21:...|               Debit|Booked|       100801|  1177 31MAR17 D , ...|2017-04-03T20:21:...|
| 32353524|{"Amount": "36.53...|{"Code":  "POS","...|2017-04-03T20:21:...|               Debit|Booked|       100802|  3818 31MAR17 , AS...|2017-04-03T20:21:...|
| 32353526|{"Amount": "4.84"...|{"Code":  "POS","...|2017-04-03T20:21:...|               Debit|Booked|       100803|  3818 31MAR17 C , ...|2017-04-03T20:21:...|
| 32353528|{"Amount": "250",...|{"

In [9]:
with open('TransformedSummary.json') as f:
        data = json.load(f)
        TransactionDf = [item for items in data.values() for item in items]
        TransactionDf=pd.DataFrame(TransactionDf)
        



In [10]:
#TransactionDf.head()

In [11]:
#print(data)

In [12]:
#print(data.values()[1])

In [14]:
TransactionDf.show()

+---------+--------------------+--------------------+--------------------+--------------------+------+-------------+----------------------+--------------------+
|AccountId|              Amount| BankTransactionCode|     BookingDateTime|CreditDebitIndicator|Status|TransactionId|TransactionInformation|       ValueDateTime|
+---------+--------------------+--------------------+--------------------+--------------------+------+-------------+----------------------+--------------------+
| 32353522|{"Amount": "6.88"...|{"Code":  "POS","...|2017-04-03T20:21:...|               Debit|Booked|       100801|  1177 31MAR17 D , ...|2017-04-03T20:21:...|
| 32353524|{"Amount": "36.53...|{"Code":  "POS","...|2017-04-03T20:21:...|               Debit|Booked|       100802|  3818 31MAR17 , AS...|2017-04-03T20:21:...|
| 32353526|{"Amount": "4.84"...|{"Code":  "POS","...|2017-04-03T20:21:...|               Debit|Booked|       100803|  3818 31MAR17 C , ...|2017-04-03T20:21:...|
| 32353528|{"Amount": "250",...|{"

In [7]:
TransactionDf['AmountInGBP']=TransactionDf['Amount'].map(lambda x :ast.literal_eval(x)['Amount']).astype(np.float)
TransactionDf['Type']=TransactionDf['BankTransactionCode'].map(lambda x :ast.literal_eval(x[:-1]+"\"\""+'}')['Code'])

In [8]:
TransactionDf.head()

,AccountId,Amount,BankTransactionCode,BookingDateTime,CreditDebitIndicator,Status,TransactionId,TransactionInformation,ValueDateTime,AmountInGBP,Type
0,32353522,"{""Amount"": ""100"", ""Currency"": ""GBP""}","{""Code"": ""OTR"","" SubCode"": }",2016-11-18T20:21:50-08:00,Credit,Booked,100211,"CALL REF.NO. 0000 , FROM A/C 10111778",2016-11-18T20:21:50-08:00,100.00,OTR
1,32353524,"{""Amount"": ""50"", ""Currency"": ""GBP""}","{""Code"": ""POS"","" SubCode"": }",2016-11-18T20:21:50-08:00,Debit,Booked,100212,"3818 17NOV16 , DEBENHAMS , CROYDON GB",2016-11-18T20:21:50-08:00,50.00,POS
2,32353526,"{""Amount"": ""2"", ""Currency"": ""GBP""}","{""Code"": ""POS"","" SubCode"": }",2016-11-23T20:21:50-08:00,Debit,Booked,100213,"3818 22NOV16 C , COOP GROUP FOOD , CROYDON GB",2016-11-23T20:21:50-08:00,2.00,POS
3,32353528,"{""Amount"": ""2.4"", ""Currency"": ""GBP""}","{""Code"": ""POS"","" SubCode"": }",2016-11-23T20:21:50-08:00,Debit,Booked,100214,"3818 22NOV16 C , TFL.GOV.UK/CP , TFL TRAVEL CH GB",2016-11-23T20:21:50-08:00,2.40,POS
4,32353530,"{""Amount"": ""4.99"", ""Currency"": ""GBP""}","{""Code"": ""POS"","" SubCode"": }",2016-11-24T20:21:50-08:00,Credit,Booked,100215,"3818 23NOV16 , LIDL UK , LONDON GB , REFUND",2016-11-24T20:21:50-08:00,4.99,POS


In [9]:
TransactionDf.columns

Index(['AccountId', 'Amount', 'BankTransactionCode', 'BookingDateTime',
       'CreditDebitIndicator', 'Status', 'TransactionId',
       'TransactionInformation', 'ValueDateTime', 'AmountInGBP', 'Type'],
      dtype='object')

In [10]:
def BinningData(TransactionDf, bins):
            min_val = min((TransactionDf['AmountInGBP'])-1)
            max_val = max(TransactionDf['AmountInGBP'])
            #print(min_val, max_val)
            custom_bucket_array = np.linspace(min_val, max_val, bins)
            cut_points = list(custom_bucket_array)
            group_name = ["low","medium","high"]
            TransactionDf["AmountInGBP_bin"] = pd.cut(TransactionDf["AmountInGBP"], cut_points, labels=group_name)
            return TransactionDf

In [11]:
 BinningData(TransactionDf, 4)

,AccountId,Amount,BankTransactionCode,BookingDateTime,CreditDebitIndicator,Status,TransactionId,TransactionInformation,ValueDateTime,AmountInGBP,Type,AmountInGBP_bin
0,32353522,"{""Amount"": ""100"", ""Currency"": ""GBP""}","{""Code"": ""OTR"","" SubCode"": }",2016-11-18T20:21:50-08:00,Credit,Booked,100211,"CALL REF.NO. 0000 , FROM A/C 10111778",2016-11-18T20:21:50-08:00,100.00,OTR,low
1,32353524,"{""Amount"": ""50"", ""Currency"": ""GBP""}","{""Code"": ""POS"","" SubCode"": }",2016-11-18T20:21:50-08:00,Debit,Booked,100212,"3818 17NOV16 , DEBENHAMS , CROYDON GB",2016-11-18T20:21:50-08:00,50.00,POS,low
2,32353526,"{""Amount"": ""2"", ""Currency"": ""GBP""}","{""Code"": ""POS"","" SubCode"": }",2016-11-23T20:21:50-08:00,Debit,Booked,100213,"3818 22NOV16 C , COOP GROUP FOOD , CROYDON GB",2016-11-23T20:21:50-08:00,2.00,POS,low
3,32353528,"{""Amount"": ""2.4"", ""Currency"": ""GBP""}","{""Code"": ""POS"","" SubCode"": }",2016-11-23T20:21:50-08:00,Debit,Booked,100214,"3818 22NOV16 C , TFL.GOV.UK/CP , TFL TRAVEL CH GB",2016-11-23T20:21:50-08:00,2.40,POS,low
4,32353530,"{""Amount"": ""4.99"", ""Currency"": ""GBP""}","{""Code"": ""POS"","" SubCode"": }",2016-11-24T20:21:50-08:00,Credit,Booked,100215,"3818 23NOV16 , LIDL UK , LONDON GB , REFUND",2016-11-24T20:21:50-08:00,4.99,POS,low
5,32353532,"{""Amount"": ""7.66"", ""Currency"": ""GBP""}","{""Code"": ""POS"","" SubCode"": }",2016-11-24T20:21:50-08:00,Debit,Booked,100216,"3818 23NOV16 C , LIDL UK , LONDON GB",2016-11-24T20:21:50-08:00,7.66,POS,low
6,32353534,"{""Amount"": ""100"", ""Currency"": ""GBP""}","{""Code"": ""OTR"","" SubCode"": }",2016-11-25T20:21:50-08:00,Credit,Booked,100217,"CALL REF.NO. 0000 , FROM A/C 10111778",2016-11-25T20:21:50-08:00,100.00,OTR,low
7,32353536,"{""Amount"": ""1"", ""Currency"": ""GBP""}","{""Code"": ""POS"","" SubCode"": }",2016-11-25T20:21:50-08:00,Debit,Booked,100218,"3818 24NOV16 C , POUNDWORLD RETAIL , CROYDON GB",2016-11-25T20:21:50-08:00,1.00,POS,low
8,32353538,"{""Amount"": ""300"", ""Currency"": ""GBP""}","{""Code"": ""OTR"","" SubCode"": }",2016-11-28T20:21:50-08:00,Credit,Booked,100219,"CALL REF.NO. 0000 , FROM A/C 10111778",2016-11-28T20:21:50-08:00,300.00,OTR,low
9,32353540,"{""Amount"": ""7.99"", ""Currency"": ""GBP""}","{""Code"": ""POS"","" SubCode"": }",2016-11-28T20:21:50-08:00,Debit,Booked,100220,"3818 26NOV16 , ARGOS LTD , CROYDON GB",2016-11-28T20:21:50-08:00,7.99,POS,low


In [12]:
def getMerchfromDesc(tranType,tranDesc):
            if tranType=='POS':        
                parsed=tranDesc.split(', ')
                lenPar=len(parsed)
                if parsed[lenPar-1] == 'REFUND':
                    merch=parsed[1:-2]
                    return ('').join(merch)+'REFUND'
                else:
                    merch=parsed[1:-1]
                    return ('').join(merch)
            else:
                return tranDesc

In [15]:
def cleanMerchant(txt):
            txt=re.sub(r'[^a-zA-Z\b]', ' ', str(txt).upper())
            return " ".join(txt.split())
TransactionDf['Merchant']=TransactionDf.apply(lambda row:getMerchfromDesc(row['Type'],row['TransactionInformation']),axis=1 )
TransactionDf['cleanedMerchant']=TransactionDf['Merchant'].map(lambda x:cleanMerchant(x))
TransactionDf['Predictorcolumn']=TransactionDf['CreditDebitIndicator']+' '+TransactionDf['cleanedMerchant']+' '+TransactionDf['Type']+' '+TransactionDf['AmountInGBP_bin'].astype(str)


In [16]:
TransactionDf.head()

,AccountId,Amount,BankTransactionCode,BookingDateTime,CreditDebitIndicator,Status,TransactionId,TransactionInformation,ValueDateTime,AmountInGBP,Type,AmountInGBP_bin,Merchant,cleanedMerchant,Predictorcolumn
0,32353522,"{""Amount"": ""100"", ""Currency"": ""GBP""}","{""Code"": ""OTR"","" SubCode"": }",2016-11-18T20:21:50-08:00,Credit,Booked,100211,"CALL REF.NO. 0000 , FROM A/C 10111778",2016-11-18T20:21:50-08:00,100.00,OTR,low,"CALL REF.NO. 0000 , FROM A/C 10111778",CALL REF NO FROM A C,Credit CALL REF NO FROM A C OTR low
1,32353524,"{""Amount"": ""50"", ""Currency"": ""GBP""}","{""Code"": ""POS"","" SubCode"": }",2016-11-18T20:21:50-08:00,Debit,Booked,100212,"3818 17NOV16 , DEBENHAMS , CROYDON GB",2016-11-18T20:21:50-08:00,50.00,POS,low,DEBENHAMS,DEBENHAMS,Debit DEBENHAMS POS low
2,32353526,"{""Amount"": ""2"", ""Currency"": ""GBP""}","{""Code"": ""POS"","" SubCode"": }",2016-11-23T20:21:50-08:00,Debit,Booked,100213,"3818 22NOV16 C , COOP GROUP FOOD , CROYDON GB",2016-11-23T20:21:50-08:00,2.00,POS,low,COOP GROUP FOOD,COOP GROUP FOOD,Debit COOP GROUP FOOD POS low
3,32353528,"{""Amount"": ""2.4"", ""Currency"": ""GBP""}","{""Code"": ""POS"","" SubCode"": }",2016-11-23T20:21:50-08:00,Debit,Booked,100214,"3818 22NOV16 C , TFL.GOV.UK/CP , TFL TRAVEL CH GB",2016-11-23T20:21:50-08:00,2.40,POS,low,TFL.GOV.UK/CP,TFL GOV UK CP,Debit TFL GOV UK CP POS low
4,32353530,"{""Amount"": ""4.99"", ""Currency"": ""GBP""}","{""Code"": ""POS"","" SubCode"": }",2016-11-24T20:21:50-08:00,Credit,Booked,100215,"3818 23NOV16 , LIDL UK , LONDON GB , REFUND",2016-11-24T20:21:50-08:00,4.99,POS,low,LIDL UK REFUND,LIDL UK REFUND,Credit LIDL UK REFUND POS low


In [17]:
TransactionDf['Merchant']=TransactionDf.apply(lambda row:getMerchfromDesc(row['Type'],row['TransactionInformation']),axis=1 )
TransactionDf['cleanedMerchant']=TransactionDf['Merchant'].map(lambda x:cleanMerchant(x))
TransactionDf['Predictorcolumn']=TransactionDf['CreditDebitIndicator']+' '+TransactionDf['cleanedMerchant']+' '+TransactionDf['Type']+' '+TransactionDf['AmountInGBP_bin'].astype(str)

In [18]:
TransactionDf.head()

,AccountId,Amount,BankTransactionCode,BookingDateTime,CreditDebitIndicator,Status,TransactionId,TransactionInformation,ValueDateTime,AmountInGBP,Type,AmountInGBP_bin,Merchant,cleanedMerchant,Predictorcolumn
0,32353522,"{""Amount"": ""100"", ""Currency"": ""GBP""}","{""Code"": ""OTR"","" SubCode"": }",2016-11-18T20:21:50-08:00,Credit,Booked,100211,"CALL REF.NO. 0000 , FROM A/C 10111778",2016-11-18T20:21:50-08:00,100.00,OTR,low,"CALL REF.NO. 0000 , FROM A/C 10111778",CALL REF NO FROM A C,Credit CALL REF NO FROM A C OTR low
1,32353524,"{""Amount"": ""50"", ""Currency"": ""GBP""}","{""Code"": ""POS"","" SubCode"": }",2016-11-18T20:21:50-08:00,Debit,Booked,100212,"3818 17NOV16 , DEBENHAMS , CROYDON GB",2016-11-18T20:21:50-08:00,50.00,POS,low,DEBENHAMS,DEBENHAMS,Debit DEBENHAMS POS low
2,32353526,"{""Amount"": ""2"", ""Currency"": ""GBP""}","{""Code"": ""POS"","" SubCode"": }",2016-11-23T20:21:50-08:00,Debit,Booked,100213,"3818 22NOV16 C , COOP GROUP FOOD , CROYDON GB",2016-11-23T20:21:50-08:00,2.00,POS,low,COOP GROUP FOOD,COOP GROUP FOOD,Debit COOP GROUP FOOD POS low
3,32353528,"{""Amount"": ""2.4"", ""Currency"": ""GBP""}","{""Code"": ""POS"","" SubCode"": }",2016-11-23T20:21:50-08:00,Debit,Booked,100214,"3818 22NOV16 C , TFL.GOV.UK/CP , TFL TRAVEL CH GB",2016-11-23T20:21:50-08:00,2.40,POS,low,TFL.GOV.UK/CP,TFL GOV UK CP,Debit TFL GOV UK CP POS low
4,32353530,"{""Amount"": ""4.99"", ""Currency"": ""GBP""}","{""Code"": ""POS"","" SubCode"": }",2016-11-24T20:21:50-08:00,Credit,Booked,100215,"3818 23NOV16 , LIDL UK , LONDON GB , REFUND",2016-11-24T20:21:50-08:00,4.99,POS,low,LIDL UK REFUND,LIDL UK REFUND,Credit LIDL UK REFUND POS low


In [19]:
with open('vectorizer.pkl','rb') as f:
            vectorizer = pickle.load(f)

In [20]:
with open('RandomForest_classifier_Feb24.pkl','rb') as f:
            RandomForest_model = pickle.load(f)

In [21]:
def predictionModule(row):
            if (row['Type']=='ATM'):
                if row['CreditDebitIndicator']=='Debit':
                    return row['Type']+' Withdrawal'
                else:
                    return row['Type']+' Deposit'
            else:
                vector=vectorizer.transform([row['Predictorcolumn']])
                prediction=RandomForest_model.predict(vector)[0]
                return prediction

In [25]:
TransactionDf['RF_Predictions']=TransactionDf.apply(lambda row:predictionModule(row),axis=1)
        #surrogate = final_default_list[['New_ID_x','final_predict','Reason']]
surrogate = TransactionDf[['AccountId','TransactionInformation','Type','RF_Predictions']]
        #final_surrogate = surrogate.drop_duplicates()        

In [26]:
dict_lst = []
for i in surrogate.iterrows():
            #print(i[1]['predict_class'])
            cust_id = i[1]['AccountId']
            Transaction_Information = i[1]['TransactionInformation']
            Transaction_Type = i[1]['Type']
            predicted_class = i[1]['RF_Predictions']
            #default_Reason = i[1]['Reason']

            elm = {"cust_id": cust_id,"Transaction_Information":Transaction_Information,"Transaction_Type":Transaction_Type,"predicted_class":predicted_class}
            dict_lst.append(elm)

In [28]:
res_data_dict = {"Transaction_Details":dict_lst}
print(res_data_dict)

{'Transaction_Details': [{'cust_id': 32353522, 'Transaction_Information': 'CALL REF.NO. 0000 , FROM A/C 10111778', 'Transaction_Type': 'OTR', 'predicted_class': 'Financial Services, Funds Transfer'}, {'cust_id': 32353524, 'Transaction_Information': '3818 17NOV16 , DEBENHAMS , CROYDON GB', 'Transaction_Type': 'POS', 'predicted_class': 'Retail, Clothing'}, {'cust_id': 32353526, 'Transaction_Information': '3818 22NOV16 C , COOP GROUP FOOD , CROYDON GB', 'Transaction_Type': 'POS', 'predicted_class': 'Retail, Supermarket'}, {'cust_id': 32353528, 'Transaction_Information': '3818 22NOV16 C , TFL.GOV.UK/CP , TFL TRAVEL CH GB', 'Transaction_Type': 'POS', 'predicted_class': 'Travel, Rail'}, {'cust_id': 32353530, 'Transaction_Information': '3818 23NOV16 , LIDL UK , LONDON GB , REFUND', 'Transaction_Type': 'POS', 'predicted_class': 'Income, Refunds'}, {'cust_id': 32353532, 'Transaction_Information': '3818 23NOV16 C , LIDL UK , LONDON GB', 'Transaction_Type': 'POS', 'predicted_class': 'Retail, Food

In [35]:
a= pd.DataFrame(res_data_dict)
a.head()

,Transaction_Details
0,"{'cust_id': 32353522, 'Transaction_Information..."
1,"{'cust_id': 32353524, 'Transaction_Information..."
2,"{'cust_id': 32353526, 'Transaction_Information..."
3,"{'cust_id': 32353528, 'Transaction_Information..."
4,"{'cust_id': 32353530, 'Transaction_Information..."
